###Very early demo of netCDF capabilities within the export module. 

In [3]:
from __future__ import print_function
import os
import flopy

Load our old friend...the Freyberg model

In [4]:
nam_file = "freyberg.nam"
model_ws = os.path.join("data")
ml = flopy.modflow.Modflow.load(nam_file,model_ws=model_ws)

Some netCDF capabilities:

In [5]:
# export a 2d array
ml.dis.top.export(os.path.join(ml.model_ws,"top.nc")

#export a 3d array
ml.lpf.hk.export("hk.nc")

# export lots of things to the same nc file
fnc = ml.dis.botm.export(os.path.join(ml.model_ws,"test.nc"))
ml.lpf.hk.export(fnc)
ml.dis.top.export(fnc)
# export transient 2d
ml.rch.rech.export(fnc)



SyntaxError: invalid syntax (<ipython-input-5-6ab648a49d33>, line 5)

In [6]:
# export whole packages
# export mflist
fnc = ml.wel.export(os.path.join(ml.model_ws,"packages.nc"))
ml.lpf.export(fnc)
fnc.nc



<type 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6
    date_created: 2015-12-11T16:54:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -27.3325004578
    geospatial_vertical_max: -17.3986301422
    geospatial_vertical_resolution: variable
    featureType: Grid
    origin_x: 0.0
    origin_y: 10000.0
    origin_crs: EPSG:4326
    grid_rotation_from_origin: 0.0
    dimensions(sizes): time(1), layer(1), y(40), x(20)
    variables(dimensions): int32 crs(), float64 time(time), float64 elevation(layer,y,x), float64 longitude(y,x), float64 latitude(y,x), float32 layer(layer), float32 delc(y), float32 delr(x), |S1 VerticalTransform(), float32 wel_flux(time,layer,y,x), float32 wel_iface(time,layer,y,x), float32 hani(layer,y,x), float32 hk(layer,y,x), float32 ss(layer,y,x), float32 sy(layer,y,x), float32 vka(layer,y,x), float32 vkcb(layer,y,x), float32 wetdry(layer,y,x)
    groups: 

In [7]:
# or just export the whole model
fnc = ml.export(os.path.join(ml.model_ws,"model.nc"))
fnc.nc

<type 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6
    date_created: 2015-12-11T16:54:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -27.3325004578
    geospatial_vertical_max: -17.3986301422
    geospatial_vertical_resolution: variable
    featureType: Grid
    origin_x: 0.0
    origin_y: 10000.0
    origin_crs: EPSG:4326
    grid_rotation_from_origin: 0.0
    dimensions(sizes): time(1), layer(1), y(40), x(20)
    variables(dimensions): int32 crs(), float64 time(time), float64 elevation(layer,y,x), float64 longitude(y,x), float64 latitude(y,x), float32 layer(layer), float32 delc(y), float32 delr(x), |S1 VerticalTransform(), float32 botm(layer,y,x), float32 thickness(layer,y,x), float32 top(y,x), float32 rech(time,y,x), int32 ibound(layer,y,x), float32 strt(layer,y,x), float32 hani(layer,y,x), float32 hk(layer,y,x), float32 ss(layer,y,x), float32 sy(layer,y,x), float32 vka(layer,y,x), float32 vkcb(layer,y,

We can also use ```.export()``` with shapefiles

In [8]:
ml.wel.export(os.path.join(ml.model_ws,"packages.shp"))
ml.export(os.path.join(ml.model_ws,"model.shp"))
ml.lpf.hk.export(os.path.join(ml.model_ws,"hk.shp"))